# Code to train models selected from brute force results

In [1]:
## imports 

import sys
import os
sys.path.insert(0, os.path.abspath('../forestpy/forestpy'))

import forest
from importlib import reload
reload(forest)
import os
from pandas import read_csv
import numpy as np

from datetime import datetime, timedelta

from multigrids import TemporalMultiGrid, TemporalGrid
import random_forest_tools as tools
reload(tools)
import matplotlib.pyplot as plt

data_dir = "/Users/rwspicer/Desktop/data/V1/"
feature_file = os.path.join(
    data_dir,
    "master-project/training/ACP/v1/temporal-multigrid/rf_traing_set_v1.yml"
)
label_file = os.path.join(
    data_dir,
    "thermokarst/initiation-regions/ACP/v3-1/PDM-5var/without_predisp/temporal-grid/ia_grid_5var.yml"
)

to_td = lambda x: timedelta(hours = int(x.split(':')[0]),minutes = int(x.split(':')[1]), seconds = float(x.split(':')[2]))
to_seconds = lambda x: int(x.split(':')[0]) *60 *60 + int(x.split(':')[1]) * 60 + float(x.split(':')[2])
to_min = lambda x: to_seconds(x) / 60
to_hour = lambda x: to_min(x) / 60


import pickle
import joblib
from pandas import DataFrame, read_csv

feature_importance_list = []
model_list = []

grid_names = ['fdd', 'tdd', 'tdd+1', 'ewp', 'fwp', 'sp', 'lsp', 'sp+1', 'lat', 'long','aspect','slope', 'elev' ]



def show_feature_importances(model, feature_list, show=False):
    # Get numerical feature importances
    importances = list(model.feature_importances_)
    # List of tuples with variable and importance
    feature_importances = [(feature, round(importance*100, 3)) for feature, importance in zip(feature_list, importances)]
    # Sort the feature importances by most important first
    feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
    # Print out the feature and importances 
    if show:
        [print('{:20} {}%'.format(*pair)) for pair in feature_importances];
        
    feature_importances = {f[0]:f[1] for f in feature_importances} 
    return feature_importances

In [2]:
## load subsampled traing data
ss_data = pickle.load(open('ss_data.pkl', 'rb'))

In [3]:
## load grid data
training_data = TemporalMultiGrid(feature_file)
om = TemporalGrid(label_file)

list_of_models = [
    'rfm_e50_md60_mfAUTO_mln50000_msl8_mss5_tdp25.yml',
    'rfm_e50_md60_mfAUTO_mln50000_msl4_mss5_tdp25.yml',
    'rfm_e50_md60_mfAUTO_mln50000_msl4_mss5_tdp25.yml',
    'rfm_e10_md60_mfAUTO_mln50000_msl8_mss5_tdp50.yml',
    'rfm_e10_md60_mfAUTO_mln50000_msl8_mss2_tdp50.yml',
    'rfm_e10_md100_mfAUTO_mln50000_msl4_mss10_tdp50.yml',
    'rfm_e100_md25_mfAUTO_mln50000_msl4_mss10_tdp25.yml',
    'rfm_e100_md25_mfAUTO_mln50000_msl2_mss5_tdp25.yml',
    'rfm_e100_md25_mfAUTO_mln50000_msl2_mss2_tdp25.yml',
    'rfm_e50_md100_mfAUTO_mln50000_msl8_mss5_tdp75.yml',
    'rfm_e50_md60_mfAUTO_mln50000_msl8_mss5_tdp75.yml',
    'rfm_e50_md60_mfAUTO_mln50000_msl2_mss10_tdp75.yml',
]

In [4]:
### this trains and saves all of the models in turn

for m in list_of_models[1:]:
    print(m)
    settings = forest.RFParams(m)
    model = forest.create_model(ss_data['tdp25']['features'], ss_data['tdp25']['labels'], settings,2,4)
    model_list.append(m)

    l = show_feature_importances(model, grid_names)
    feature_importance_list.append(l)
    
    joblib.dump(model,  '/Volumes/toshi-stati/tk-rf-models/' + m.split('.')[0]+'.joblib') 

rfm_e50_md60_mfAUTO_mln50000_msl4_mss5_tdp25.yml


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 50building tree 2 of 50building tree 3 of 50


building tree 4 of 50


KeyboardInterrupt: 